In [2]:
import pandas as pd
import sqlalchemy as sql
from sql_functions import get_sql_config
from sql_functions import get_engine
from sql_functions import push_to_database
from update import update_sql_table
import psycopg2

csv datei einlesen

In [3]:
sachsen_22 = pd.read_csv("data/sachsen_22.csv", delimiter= ";", encoding= 'windows-1252',dtype={'Titel':str})
sachsen_22.head(20)
sachsen_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6677 entries, 0 to 6676
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Einzelplan         6677 non-null   int64  
 1   Bezeichnung        6677 non-null   object 
 2   Kapitel            6677 non-null   int64  
 3   Bezeichnung.1      6677 non-null   object 
 4   Einnahme/Ausgabe   6677 non-null   object 
 5   TG                 2698 non-null   float64
 6   Bezeichnung.2      2698 non-null   object 
 7   Titel              6677 non-null   object 
 8   FKZ                6677 non-null   int64  
 9   Bezeichnung FKZ    6677 non-null   object 
 10  Zweckbestimmung    6677 non-null   object 
 11  Ansatz 2021 in T€  5165 non-null   object 
 12  Ansatz 2022 in T€  5131 non-null   object 
dtypes: float64(1), int64(3), object(9)
memory usage: 678.3+ KB


In [4]:
sachsen_22.head(20)

,Einzelplan,Bezeichnung,Kapitel,Bezeichnung.1,Einnahme/Ausgabe,TG,Bezeichnung.2,Titel,FKZ,Bezeichnung FKZ,Zweckbestimmung,Ansatz 2021 in T€,Ansatz 2022 in T€
0,1,Landtag,101,Landtag,Einnahme,NaN,NaN,11910,11,Politische Führung,Einnahmen aus Veröffentlichungen,NaN,NaN
1,1,Landtag,101,Landtag,Einnahme,NaN,NaN,11949,11,Politische Führung,Vermischte Einnahmen,"2,0","2,0"
2,1,Landtag,101,Landtag,Einnahme,NaN,NaN,13201,11,Politische Führung,Einnahmen aus der Veräußerung von beweglichen ...,"1,0","1,0"
3,1,Landtag,101,Landtag,Einnahme,NaN,NaN,28108,850,Rücklagen,Erstattungen des Generationenfonds,"0,0","0,0"
4,1,Landtag,101,Landtag,Ausgabe,NaN,NaN,41103,11,Politische Führung,Grundentschädigung nach \n§ 5 SächsAbgG,"9.132,5","9.570,2"
5,1,Landtag,101,Landtag,Ausgabe,NaN,NaN,41104,11,Politische Führung,Aufwandsentschädigung nach \n§ 6 SächsAbgG,"6.049,5","6.109,4"
6,1,Landtag,101,Landtag,Ausgabe,NaN,NaN,41105,11,Politische Führung,Dienstreisekosten nach § 11 SächsAbgG einschli...,"182,0","182,0"
7,1,Landtag,101,Landtag,Ausgabe,NaN,NaN,41106,11,Politische Führung,"Leistungen nach §§ 10, 11a, 21 und 22 SächsAbgG","627,9","540,2"
8,1,Landtag,101,Landtag,Ausgabe,NaN,NaN,41107,11,Politische Führung,Aufwandsentschädigung für Kosten der Ausstattu...,"417,4",NaN
9,1,Landtag,101,Landtag,Ausgabe,NaN,NaN,41108,11,Politische Führung,Leistungen an ausscheidende und ehemalige Mitg...,"6.738,9","7.633,0"


columns struktur anpassen

In [5]:
sachsen_22.columns = sachsen_22.columns.str.replace(" ", "_")
sachsen_22.columns = sachsen_22.columns.str.lower()
sachsen_22.head()

,einzelplan,bezeichnung,kapitel,bezeichnung.1,einnahme/ausgabe,tg,bezeichnung.2,titel,fkz,bezeichnung_fkz,zweckbestimmung,ansatz_2021_in_t€,ansatz_2022_in_t€
0,1,Landtag,101,Landtag,Einnahme,NaN,NaN,11910,11,Politische Führung,Einnahmen aus Veröffentlichungen,NaN,NaN
1,1,Landtag,101,Landtag,Einnahme,NaN,NaN,11949,11,Politische Führung,Vermischte Einnahmen,"2,0","2,0"
2,1,Landtag,101,Landtag,Einnahme,NaN,NaN,13201,11,Politische Führung,Einnahmen aus der Veräußerung von beweglichen ...,"1,0","1,0"
3,1,Landtag,101,Landtag,Einnahme,NaN,NaN,28108,850,Rücklagen,Erstattungen des Generationenfonds,"0,0","0,0"
4,1,Landtag,101,Landtag,Ausgabe,NaN,NaN,41103,11,Politische Führung,Grundentschädigung nach \n§ 5 SächsAbgG,"9.132,5","9.570,2"


In [6]:
sachsen_22.rename(columns = {'einzelplan':'ep' }, inplace = True)
sachsen_22.head()

,ep,bezeichnung,kapitel,bezeichnung.1,einnahme/ausgabe,tg,bezeichnung.2,titel,fkz,bezeichnung_fkz,zweckbestimmung,ansatz_2021_in_t€,ansatz_2022_in_t€
0,1,Landtag,101,Landtag,Einnahme,NaN,NaN,11910,11,Politische Führung,Einnahmen aus Veröffentlichungen,NaN,NaN
1,1,Landtag,101,Landtag,Einnahme,NaN,NaN,11949,11,Politische Führung,Vermischte Einnahmen,"2,0","2,0"
2,1,Landtag,101,Landtag,Einnahme,NaN,NaN,13201,11,Politische Führung,Einnahmen aus der Veräußerung von beweglichen ...,"1,0","1,0"
3,1,Landtag,101,Landtag,Einnahme,NaN,NaN,28108,850,Rücklagen,Erstattungen des Generationenfonds,"0,0","0,0"
4,1,Landtag,101,Landtag,Ausgabe,NaN,NaN,41103,11,Politische Führung,Grundentschädigung nach \n§ 5 SächsAbgG,"9.132,5","9.570,2"


Columns die nicht gebraucht werden droppen

In [7]:
sachsen_22.drop('ansatz_2021_in_t€', inplace=True, axis=1)


In [8]:
sachsen_22.drop('tg', inplace=True, axis=1)

In [9]:
sachsen_22.head()

,ep,bezeichnung,kapitel,bezeichnung.1,einnahme/ausgabe,bezeichnung.2,titel,fkz,bezeichnung_fkz,zweckbestimmung,ansatz_2022_in_t€
0,1,Landtag,101,Landtag,Einnahme,NaN,11910,11,Politische Führung,Einnahmen aus Veröffentlichungen,NaN
1,1,Landtag,101,Landtag,Einnahme,NaN,11949,11,Politische Führung,Vermischte Einnahmen,"2,0"
2,1,Landtag,101,Landtag,Einnahme,NaN,13201,11,Politische Führung,Einnahmen aus der Veräußerung von beweglichen ...,"1,0"
3,1,Landtag,101,Landtag,Einnahme,NaN,28108,850,Rücklagen,Erstattungen des Generationenfonds,"0,0"
4,1,Landtag,101,Landtag,Ausgabe,NaN,41103,11,Politische Führung,Grundentschädigung nach \n§ 5 SächsAbgG,"9.570,2"


Columns umbenennen

In [10]:
sachsen_22.rename(columns={'ansatz_2022_in_t€':'ansatz_2022_in_euro'}, inplace=True)

In [11]:
sachsen_22.tail()

,ep,bezeichnung,kapitel,bezeichnung.1,einnahme/ausgabe,bezeichnung.2,titel,fkz,bezeichnung_fkz,zweckbestimmung,ansatz_2022_in_euro
6672,15,Allgemeine Finanzverwaltung,1540,Versorgung und Beihilfen,Ausgabe,NaN,63202,219,Sonstige Verwaltung für soziale Angelegenheiten,Erstattungen zur Erfüllung von Verbindlichkeit...,"9,6"
6673,15,Allgemeine Finanzverwaltung,1540,Versorgung und Beihilfen,Ausgabe,NaN,63301,18,Versorgung einschließlich Beihilfen für Versor...,Erstattungen von Versorgungsanteilen an Gemein...,"1.250,0"
6674,15,Allgemeine Finanzverwaltung,1540,Versorgung und Beihilfen,Ausgabe,NaN,67101,18,Versorgung einschließlich Beihilfen für Versor...,"Versorgungszuschläge bei Beurlaubungen, Erstat...","50,0"
6675,15,Allgemeine Finanzverwaltung,1540,Versorgung und Beihilfen,Ausgabe,NaN,68520,850,Rücklagen,Zuführungen an den Generationenfonds,"30.000,0"
6676,15,Allgemeine Finanzverwaltung,1540,Versorgung und Beihilfen,Ausgabe,NaN,68521,850,Rücklagen,Zuführungen an den Generationenfonds,NaN


erneut ein column gedropped

In [12]:
sachsen_22.drop('bezeichnung.2', inplace=True, axis=1)


In [13]:
sachsen_22.head()

,ep,bezeichnung,kapitel,bezeichnung.1,einnahme/ausgabe,titel,fkz,bezeichnung_fkz,zweckbestimmung,ansatz_2022_in_euro
0,1,Landtag,101,Landtag,Einnahme,11910,11,Politische Führung,Einnahmen aus Veröffentlichungen,NaN
1,1,Landtag,101,Landtag,Einnahme,11949,11,Politische Führung,Vermischte Einnahmen,"2,0"
2,1,Landtag,101,Landtag,Einnahme,13201,11,Politische Führung,Einnahmen aus der Veräußerung von beweglichen ...,"1,0"
3,1,Landtag,101,Landtag,Einnahme,28108,850,Rücklagen,Erstattungen des Generationenfonds,"0,0"
4,1,Landtag,101,Landtag,Ausgabe,41103,11,Politische Führung,Grundentschädigung nach \n§ 5 SächsAbgG,"9.570,2"


NaN Werte in 0 umgewandelt

In [14]:
sachsen_22["ansatz_2022_in_euro"] = sachsen_22["ansatz_2022_in_euro"].fillna('0,0')

In [15]:
sachsen_22.head()

,ep,bezeichnung,kapitel,bezeichnung.1,einnahme/ausgabe,titel,fkz,bezeichnung_fkz,zweckbestimmung,ansatz_2022_in_euro
0,1,Landtag,101,Landtag,Einnahme,11910,11,Politische Führung,Einnahmen aus Veröffentlichungen,"0,0"
1,1,Landtag,101,Landtag,Einnahme,11949,11,Politische Führung,Vermischte Einnahmen,"2,0"
2,1,Landtag,101,Landtag,Einnahme,13201,11,Politische Führung,Einnahmen aus der Veräußerung von beweglichen ...,"1,0"
3,1,Landtag,101,Landtag,Einnahme,28108,850,Rücklagen,Erstattungen des Generationenfonds,"0,0"
4,1,Landtag,101,Landtag,Ausgabe,41103,11,Politische Führung,Grundentschädigung nach \n§ 5 SächsAbgG,"9.570,2"


Ep (einzeplan) angepasst und vereinheitlicht

In [16]:
sachsen_22['ep'] = sachsen_22['ep'].astype(str)
sachsen_22['ep'] = sachsen_22['ep'].apply(lambda x:x.zfill(2))

In [17]:
sachsen_22.head()

,ep,bezeichnung,kapitel,bezeichnung.1,einnahme/ausgabe,titel,fkz,bezeichnung_fkz,zweckbestimmung,ansatz_2022_in_euro
0,01,Landtag,101,Landtag,Einnahme,11910,11,Politische Führung,Einnahmen aus Veröffentlichungen,"0,0"
1,01,Landtag,101,Landtag,Einnahme,11949,11,Politische Führung,Vermischte Einnahmen,"2,0"
2,01,Landtag,101,Landtag,Einnahme,13201,11,Politische Führung,Einnahmen aus der Veräußerung von beweglichen ...,"1,0"
3,01,Landtag,101,Landtag,Einnahme,28108,850,Rücklagen,Erstattungen des Generationenfonds,"0,0"
4,01,Landtag,101,Landtag,Ausgabe,41103,11,Politische Führung,Grundentschädigung nach \n§ 5 SächsAbgG,"9.570,2"


State-ID hinzugefügt

In [18]:
sachsen_22.insert(loc=10,
          column='state_id',
          value=13)
sachsen_22.insert(loc=11,
            column='year',
            value=2022)

In [19]:
sachsen_22.head()

,ep,bezeichnung,kapitel,bezeichnung.1,einnahme/ausgabe,titel,fkz,bezeichnung_fkz,zweckbestimmung,ansatz_2022_in_euro,state_id,year
0,01,Landtag,101,Landtag,Einnahme,11910,11,Politische Führung,Einnahmen aus Veröffentlichungen,"0,0",13,2022
1,01,Landtag,101,Landtag,Einnahme,11949,11,Politische Führung,Vermischte Einnahmen,"2,0",13,2022
2,01,Landtag,101,Landtag,Einnahme,13201,11,Politische Führung,Einnahmen aus der Veräußerung von beweglichen ...,"1,0",13,2022
3,01,Landtag,101,Landtag,Einnahme,28108,850,Rücklagen,Erstattungen des Generationenfonds,"0,0",13,2022
4,01,Landtag,101,Landtag,Ausgabe,41103,11,Politische Führung,Grundentschädigung nach \n§ 5 SächsAbgG,"9.570,2",13,2022


Struktur ders Columns strukturiert und umgewandelt in einem float und *1000 multipliziert

In [20]:
sachsen_22.ansatz_2022_in_euro = sachsen_22['ansatz_2022_in_euro'].str.replace('.', '').str.replace(",", ".").astype(float)
sachsen_22.ansatz_2022_in_euro = sachsen_22['ansatz_2022_in_euro'].apply(lambda x:int(x*1000))

/var/folders/rp/8t70135j493dk68133cwmvx40000gn/T/ipykernel_4800/3615814947.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  sachsen_22.ansatz_2022_in_euro = sachsen_22['ansatz_2022_in_euro'].str.replace('.', '').str.replace(",", ".").astype(float)


In [21]:
sachsen_22.head()

,ep,bezeichnung,kapitel,bezeichnung.1,einnahme/ausgabe,titel,fkz,bezeichnung_fkz,zweckbestimmung,ansatz_2022_in_euro,state_id,year
0,01,Landtag,101,Landtag,Einnahme,11910,11,Politische Führung,Einnahmen aus Veröffentlichungen,0,13,2022
1,01,Landtag,101,Landtag,Einnahme,11949,11,Politische Führung,Vermischte Einnahmen,2000,13,2022
2,01,Landtag,101,Landtag,Einnahme,13201,11,Politische Führung,Einnahmen aus der Veräußerung von beweglichen ...,1000,13,2022
3,01,Landtag,101,Landtag,Einnahme,28108,850,Rücklagen,Erstattungen des Generationenfonds,0,13,2022
4,01,Landtag,101,Landtag,Ausgabe,41103,11,Politische Führung,Grundentschädigung nach \n§ 5 SächsAbgG,9570200,13,2022


In [22]:
sachsen_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6677 entries, 0 to 6676
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ep                   6677 non-null   object
 1   bezeichnung          6677 non-null   object
 2   kapitel              6677 non-null   int64 
 3   bezeichnung.1        6677 non-null   object
 4   einnahme/ausgabe     6677 non-null   object
 5   titel                6677 non-null   object
 6   fkz                  6677 non-null   int64 
 7   bezeichnung_fkz      6677 non-null   object
 8   zweckbestimmung      6677 non-null   object
 9   ansatz_2022_in_euro  6677 non-null   int64 
 10  state_id             6677 non-null   int64 
 11  year                 6677 non-null   int64 
dtypes: int64(5), object(7)
memory usage: 626.1+ KB


Columns umgewandelt und strukturiert 

In [23]:
sachsen_22.titel=sachsen_22.titel.astype(str)

In [24]:
sachsen_22['gruppe']=sachsen_22.titel.str[:3]

In [25]:
sachsen_22['counter']=sachsen_22.titel.str[3:]

In [26]:
sachsen_22['ansatz_2022_in_euro'] =sachsen_22[['ansatz_2022_in_euro', 'gruppe']].apply(lambda x:-x[0] if int(x[1]) > 400 else x[0], axis=1)

In [27]:
sachsen_22.head()

,ep,bezeichnung,kapitel,bezeichnung.1,einnahme/ausgabe,titel,fkz,bezeichnung_fkz,zweckbestimmung,ansatz_2022_in_euro,state_id,year,gruppe,counter
0,01,Landtag,101,Landtag,Einnahme,11910,11,Politische Führung,Einnahmen aus Veröffentlichungen,0,13,2022,119,10
1,01,Landtag,101,Landtag,Einnahme,11949,11,Politische Führung,Vermischte Einnahmen,2000,13,2022,119,49
2,01,Landtag,101,Landtag,Einnahme,13201,11,Politische Führung,Einnahmen aus der Veräußerung von beweglichen ...,1000,13,2022,132,01
3,01,Landtag,101,Landtag,Einnahme,28108,850,Rücklagen,Erstattungen des Generationenfonds,0,13,2022,281,08
4,01,Landtag,101,Landtag,Ausgabe,41103,11,Politische Führung,Grundentschädigung nach \n§ 5 SächsAbgG,-9570200,13,2022,411,03


columns einnahme/ausgabe und titel gedropped

In [28]:
sachsen_22.drop('einnahme/ausgabe', inplace=True, axis=1)
sachsen_22.drop('titel', inplace=True, axis=1)

In [29]:
sachsen_22.head()

,ep,bezeichnung,kapitel,bezeichnung.1,fkz,bezeichnung_fkz,zweckbestimmung,ansatz_2022_in_euro,state_id,year,gruppe,counter
0,01,Landtag,101,Landtag,11,Politische Führung,Einnahmen aus Veröffentlichungen,0,13,2022,119,10
1,01,Landtag,101,Landtag,11,Politische Führung,Vermischte Einnahmen,2000,13,2022,119,49
2,01,Landtag,101,Landtag,11,Politische Führung,Einnahmen aus der Veräußerung von beweglichen ...,1000,13,2022,132,01
3,01,Landtag,101,Landtag,850,Rücklagen,Erstattungen des Generationenfonds,0,13,2022,281,08
4,01,Landtag,101,Landtag,11,Politische Führung,Grundentschädigung nach \n§ 5 SächsAbgG,-9570200,13,2022,411,03


Kapitel neu strukturiert und verinheitlicht

In [30]:
sachsen_22['kapitel'] = sachsen_22['kapitel'].astype(str)
sachsen_22['kapitel'] = sachsen_22['kapitel'].str[-2:]

In [31]:
sachsen_22.tail()

,ep,bezeichnung,kapitel,bezeichnung.1,fkz,bezeichnung_fkz,zweckbestimmung,ansatz_2022_in_euro,state_id,year,gruppe,counter
6672,15,Allgemeine Finanzverwaltung,40,Versorgung und Beihilfen,219,Sonstige Verwaltung für soziale Angelegenheiten,Erstattungen zur Erfüllung von Verbindlichkeit...,-9600,13,2022,632,02
6673,15,Allgemeine Finanzverwaltung,40,Versorgung und Beihilfen,18,Versorgung einschließlich Beihilfen für Versor...,Erstattungen von Versorgungsanteilen an Gemein...,-1250000,13,2022,633,01
6674,15,Allgemeine Finanzverwaltung,40,Versorgung und Beihilfen,18,Versorgung einschließlich Beihilfen für Versor...,"Versorgungszuschläge bei Beurlaubungen, Erstat...",-50000,13,2022,671,01
6675,15,Allgemeine Finanzverwaltung,40,Versorgung und Beihilfen,850,Rücklagen,Zuführungen an den Generationenfonds,-30000000,13,2022,685,20
6676,15,Allgemeine Finanzverwaltung,40,Versorgung und Beihilfen,850,Rücklagen,Zuführungen an den Generationenfonds,0,13,2022,685,21


In [32]:
sachsen_22['kapitel'] = sachsen_22['kapitel'].apply(lambda x:x.zfill(3))
sachsen_22['fkz'] = sachsen_22['fkz'].astype(str)
sachsen_22['fkz'] = sachsen_22['fkz'].apply(lambda x:x.zfill(3))

In [33]:
sachsen_22.head()


,ep,bezeichnung,kapitel,bezeichnung.1,fkz,bezeichnung_fkz,zweckbestimmung,ansatz_2022_in_euro,state_id,year,gruppe,counter
0,01,Landtag,001,Landtag,011,Politische Führung,Einnahmen aus Veröffentlichungen,0,13,2022,119,10
1,01,Landtag,001,Landtag,011,Politische Führung,Vermischte Einnahmen,2000,13,2022,119,49
2,01,Landtag,001,Landtag,011,Politische Führung,Einnahmen aus der Veräußerung von beweglichen ...,1000,13,2022,132,01
3,01,Landtag,001,Landtag,850,Rücklagen,Erstattungen des Generationenfonds,0,13,2022,281,08
4,01,Landtag,001,Landtag,011,Politische Führung,Grundentschädigung nach \n§ 5 SächsAbgG,-9570200,13,2022,411,03


Neue DF erstellen

In [34]:
fkz_index_df = sachsen_22[['fkz','bezeichnung_fkz']].copy()

In [35]:
fkz_index_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6677 entries, 0 to 6676
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   fkz              6677 non-null   object
 1   bezeichnung_fkz  6677 non-null   object
dtypes: object(2)
memory usage: 104.5+ KB


In [36]:

#fkz_index_df[fkz_index_df.duplicated(['fkz', 'bezeichnung_fkz'], keep=False)]
fkz_index_df.drop_duplicates(inplace=True)

In [37]:
fkz_index_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 138 entries, 0 to 6564
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   fkz              138 non-null    object
 1   bezeichnung_fkz  138 non-null    object
dtypes: object(2)
memory usage: 3.2+ KB


In [38]:
sachsen_22

,ep,bezeichnung,kapitel,bezeichnung.1,fkz,bezeichnung_fkz,zweckbestimmung,ansatz_2022_in_euro,state_id,year,gruppe,counter
0,01,Landtag,001,Landtag,011,Politische Führung,Einnahmen aus Veröffentlichungen,0,13,2022,119,10
1,01,Landtag,001,Landtag,011,Politische Führung,Vermischte Einnahmen,2000,13,2022,119,49
2,01,Landtag,001,Landtag,011,Politische Führung,Einnahmen aus der Veräußerung von beweglichen ...,1000,13,2022,132,01
3,01,Landtag,001,Landtag,850,Rücklagen,Erstattungen des Generationenfonds,0,13,2022,281,08
4,01,Landtag,001,Landtag,011,Politische Führung,Grundentschädigung nach \n§ 5 SächsAbgG,-9570200,13,2022,411,03
...,...,...,...,...,...,...,...,...,...,...,...,...
6672,15,Allgemeine Finanzverwaltung,040,Versorgung und Beihilfen,219,Sonstige Verwaltung für soziale Angelegenheiten,Erstattungen zur Erfüllung von Verbindlichkeit...,-9600,13,2022,632,02
6673,15,Allgemeine Finanzverwaltung,040,Versorgung und Beihilfen,018,Versorgung einschließlich Beihilfen für Versor...,Erstattungen von Versorgungsanteilen an Gemein...,-1250000,13,2022,633,01
6674,15,Allgemeine Finanzverwaltung,040,Versorgung und Beihilfen,018,Versorgung einschließlich Beihilfen für Versor...,"Versorgungszuschläge bei Beurlaubungen, Erstat...",-50000,13,2022,671,01
6675,15,Allgemeine Finanzverwaltung,040,Versorgung und Beihilfen,850,Rücklagen,Zuführungen an den Generationenfonds,-30000000,13,2022,685,20


In [39]:
sachsen_ep = sachsen_22[['ep','bezeichnung', 'state_id', 'year']].copy()
sachsen_ep.rename(columns={'bezeichnung':'ep_txt'}, inplace=True)
sachsen_ep.drop_duplicates(inplace=True)

sachsen_kapitel = sachsen_22[['ep','kapitel', 'bezeichnung.1', 'state_id', 'year']].copy()
sachsen_kapitel.rename(columns={'bezeichnung.1':'kapitel_txt'}, inplace=True)
sachsen_kapitel.drop_duplicates(inplace=True)

sachsen_22 = sachsen_22.drop(['bezeichnung', 'bezeichnung.1', 'bezeichnung_fkz'], axis=1)
sachsen_22.rename(columns={'ansatz_2022_in_euro':'amount'}, inplace=True)

sachsen_22

,ep,kapitel,fkz,zweckbestimmung,amount,state_id,year,gruppe,counter
0,01,001,011,Einnahmen aus Veröffentlichungen,0,13,2022,119,10
1,01,001,011,Vermischte Einnahmen,2000,13,2022,119,49
2,01,001,011,Einnahmen aus der Veräußerung von beweglichen ...,1000,13,2022,132,01
3,01,001,850,Erstattungen des Generationenfonds,0,13,2022,281,08
4,01,001,011,Grundentschädigung nach \n§ 5 SächsAbgG,-9570200,13,2022,411,03
...,...,...,...,...,...,...,...,...,...
6672,15,040,219,Erstattungen zur Erfüllung von Verbindlichkeit...,-9600,13,2022,632,02
6673,15,040,018,Erstattungen von Versorgungsanteilen an Gemein...,-1250000,13,2022,633,01
6674,15,040,018,"Versorgungszuschläge bei Beurlaubungen, Erstat...",-50000,13,2022,671,01
6675,15,040,850,Zuführungen an den Generationenfonds,-30000000,13,2022,685,20


In [40]:
sachsen_ep

,ep,ep_txt,state_id,year
0,01,Landtag,13,2022
97,02,Staatskanzlei,13,2022
380,03,Staatsministerium des Innern,13,2022
1463,04,Staatsministerium der Finanzen,13,2022
1608,05,Staatsministerium für Kultus,13,2022
2597,06,Staatsministerium der Justiz und für Demokrati...,13,2022
3214,07,"Staatsministerium für Wirtschaft, Arbeit und V...",13,2022
3796,08,Staatsministerium für Soziales und Gesellschaf...,13,2022
4324,09,"Staatsministerium für Energie, Klimaschutz, Um...",13,2022
4887,10,Staatsministerium für Regionalentwicklung,13,2022


push it to the limit

In [40]:
get_sql_config()
engine = get_engine()

In [41]:
table_name = 'einzelplaene'
update_sql_table(sachsen_ep,table_name)
#push_to_database(sachsen_ep, "einzelplaene", engine, "capstone_public_budgeting")


In [41]:
table_name = 'kapitel'
update_sql_table(sachsen_kapitel,table_name)
#push_to_database(sachsen_kapitel, "kapitel", engine, "capstone_public_budgeting")

The kapitel table was imported successfully.


In [44]:
#push_to_database(sachsen_22, "budget_all", engine, "capstone_public_budgeting")

The budget_all table was imported successfully.
